In [2]:
import ee
import geemap
from gee_0_utils import *

initialize()
config = ProjectConfig()
roi = config.roi
data_folder = config.data_folder

def fc_to_image(fc, property, scale = 1000):
    # Convert the feature collection to an image
    return fc.reduceToImage(
            properties = [property],
            reducer = ee.Reducer.first()
        ).reproject(
            crs = 'EPSG:4326',  # or match your source CRS
            scale = scale
        ).rename(property)


def export_image(image, description):
    task = ee.batch.Export.image.toDrive(
        image = image,
        description = description,
        fileNamePrefix = description,
        region = roi,
        scale = image.projection().nominalScale(),
        maxPixels = 1e13,
        crs = 'EPSG:4326',
        fileFormat = 'GeoTIFF'
    )
    task.start()

In [4]:
lulc = (ee.Image("projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_integration_v1")
            .select([f"classification_{year}" for year in config.range_1985_2020])
            .byte()
            .rename([str(year) for year in config.range_1985_2020]))

pastureland = lulc.select("2020").eq(15).unmask(0).rename("pastureland")

map = geemap.Map()
map.addLayer(pastureland, {"palette": ["green", "yellow"]}, "Pastureland 2020")
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Export asymptotes for Figure 2

In [ ]:
nearest_mature = ee.Image(f"{data_folder}/nearest_mature").selfMask()

quarters_ecoreg_biomass = ee.Image(f"{data_folder}/quarters_ecoreg_biomass").select("quarter_biomass").selfMask()

biomes = ee.Image(f"{data_folder}/categorical").select("biome")
biomes_mask = biomes.eq(1).rename("biome_mask").selfMask()

# nearest_mature = ee.Image(f"{data_folder}/nearest_mature").selfMask()


# export_image(nearest_mature, "nearest_mature")
# export_image(quarters_ecoreg_biomass, "quarters_ecoreg_biomass")
# export_image(biomes_mask, "biome_mask")

## Export predictions for Figure 4

In [45]:

# Predictions for 2050
pred_lag_2050_fc = ee.FeatureCollection(f"{data_folder}/pred_lag_2050_secondary")
# multiply the value of pred by 1000
pred_lag_2050 = fc_to_image(pred_lag_2050_fc, "pred", scale=100)

# pred_lag_2050_10k = fc_to_image(pred_lag_2050_fc, "pred", scale=10000)

# Aggregate the high-resolution pixels into the 10 km grid
pred_lag_2050_10k = pred_lag_2050.reduceResolution(
    reducer = ee.Reducer.median(),
    maxPixels = 65535,
    bestEffort = True
).reproject(
    crs = 'EPSG:4326',
    scale = 10000
).rename("pred_lag_2050_10k")

vis_params = {'min': 0, 'max': 75, 'palette': ["#003f5c", "#2f4b7c", "#665191", "#a05195",
    "#d45087", "#f95d6a", "#ff7c43", "#ffa600", "#ffc300", "#ffda6a"]}

map = geemap.Map(basemap = 'CartoDB.Positron')
# map.addLayer(pred_lag_2050_fc, {}, "pred_lag_2050_fc")
map.addLayer(pred_lag_2050, vis_params, "biomass_2050")
map.addLayer(pred_lag_2050_10k, vis_params, "pred_lag_2050_10k")
map.add_colorbar(vis_params, label = "biomass (Mg/ha)", layer_name = "biomass")
map

export_image(pred_lag_2050_10k, "pred_lag_2050_secondary")

In [51]:

# Predictions for 2050
pred_lag_2050_fc = ee.FeatureCollection(f"{data_folder}/pred_lag_2050_pastureland_all")
# multiply the value of pred by 1000
pred_lag_2050 = fc_to_image(pred_lag_2050_fc, "pred", scale=100)

# multiply value by 1000
# pred_lag_2050 = pred_lag_2050.multiply(1000)

# Aggregate the high-resolution pixels into the 10 km grid
pred_lag_2050_10k = pred_lag_2050.reduceResolution(
    reducer = ee.Reducer.median(),
    maxPixels = 65535,
    bestEffort = True
).reproject(
    crs = 'EPSG:4326',
    scale = 10000
).rename("pred_lag_2050_10k")


vis_params = {'min': 0, 'max': 75, 'palette': ["#003f5c", "#2f4b7c", "#665191", "#a05195",
    "#d45087", "#f95d6a", "#ff7c43", "#ffa600", "#ffc300", "#ffda6a"]}

map = geemap.Map() #basemap = 'CartoDB.Positron'
# map.addLayer(pred_lag_2050_fc, {}, "pred_lag_2050_fc")
map.addLayer(pred_lag_2050, vis_params, "biomass_2050")
map.addLayer(pred_lag_2050_10k, vis_params, "pred_lag_2050_10k")
map.add_colorbar(vis_params, label = "biomass (Mg/ha)", layer_name = "biomass")
map

export_image(pred_lag_2050_10k, "pred_lag_2050_pastureland")

## Export Extended Data Maps

In [ ]:
# Absolute error
relative_growth = ee.FeatureCollection(f"{data_folder}/results/relative_growth")
relative_growth = fc_to_image(relative_growth, "percentage")

vis_params = {'min': 40, 'max': 60, 'palette': ['yellow', 'blue']}

map = geemap.Map(basemap = 'CartoDB.Positron')
map.addLayer(relative_growth, vis_params, "relative_growth")
map.add_colorbar(vis_params, label = "Percentage of asymptote", layer_name = "relative_growth")
map



Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
# Absolute error
absolute_error = ee.FeatureCollection(f"{data_folder}/results/absolute_error")
absolute_error = fc_to_image(absolute_error, "error")

# Percent error
percent_error = ee.FeatureCollection(f"{data_folder}/results/percent_error")
percent_error = fc_to_image(percent_error, "error")

vis_params = {'min': -100, 'max': 100, 'palette': ['red', 'blue']}

map = geemap.Map(basemap = 'CartoDB.Positron')
map.addLayer(absolute_error, vis_params, "absolute_error")
map.addLayer(percent_error, vis_params, "percent_error")
map.add_colorbar(vis_params, label = "Absolute error", layer_name = "absolute_error")
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…